In [37]:
import os
import sys
import pandas as pd
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

<h1>Loading database</h1>

In [38]:
from pypots.benchpots.datasets import preprocess_physionet2012
physionet2012_dataset = preprocess_physionet2012(subset="all", rate=0.1)

2024-10-30 11:12:52 [INFO]: You're using dataset physionet_2012, please cite it properly in your work. You can find its reference information at the below link: 
https://github.com/WenjieDu/TSDB/tree/main/dataset_profiles/physionet_2012
2024-10-30 11:12:52 [INFO]: Dataset physionet_2012 has already been downloaded. Processing directly...
2024-10-30 11:12:52 [INFO]: Dataset physionet_2012 has already been cached. Loading from cache directly...
2024-10-30 11:12:52 [INFO]: Loaded successfully!


<h1>Training data</h1>

<h3>Loading training database</h3>

In [39]:
train_X = physionet2012_dataset['train_X']

<h3>Divided into subgroups by gender and showing the percentage</h3>

In [40]:
distribution_gender_training = train_X['Gender'].value_counts(normalize=True) * 100
distribution_gender_training # 0: female, or 1: male

Gender
 1.0    56.120454
 0.0    43.801330
-1.0     0.078217
Name: proportion, dtype: float64

<h3>Divided into subgroups by ICUType and showing the percentage</h3>

In [41]:
distribution_ICUType_training = train_X['ICUType'].value_counts(normalize=True) * 100
distribution_ICUType_training

ICUType
3.0    35.575544
4.0    28.483900
2.0    21.079390
1.0    14.861165
Name: proportion, dtype: float64

<h3>Filtering only one age per patient</h3>

In [42]:
age_uniques_train = train_X.groupby('RecordID')['Age'].first().reset_index()
age_uniques_train.head()

,RecordID,Age
0,132539,54.0
1,132543,68.0
2,132547,64.0
3,132548,68.0
4,132551,78.0


<h3>Showing total occurrences of age</h3>

In [43]:
# 60% de 11988 (training set)
age_uniques_train.count()

RecordID    7671
Age         7671
dtype: int64

In [44]:
#first_values = train_X['Age'][::48] #first value of "Age" at every 48th index
#total_quantity_of_age_occurrences = first_values.count()
#total_quantity_of_age_occurrences

In [45]:
#first_values.head()

<h3>Dividing age into subgroup of people aged 65+</h3>

In [46]:
more_than_or_equal_to_65_training = age_uniques_train[age_uniques_train['Age'] >= 65]
# more_than_or_equal_to_65 = train_X[train_X['Age'] >= 65] 
#distribution_age_group_one = more_than_or_equal_to_65['Age'].count() 
#distribution_age_group_one
# age_uniques_more_than_or_equal_to_65 = more_than_or_equal_to_65.groupby('RecordID')['Age'].first().reset_index()
percentage_group_one = (more_than_or_equal_to_65_training.count() / age_uniques_train.count())*100
percentage_group_one

RecordID    54.895059
Age         54.895059
dtype: float64

<h3>Dividing age into subgroup of people under 65 years of age</h3>

In [47]:
less_than_65 = age_uniques_train[age_uniques_train['Age'] < 65]
# less_than_65 = train_X[train_X['Age'] < 65] 
#distribution_age_group_two = less_than_65['Age'].value_counts(normalize=True) * 100
#distribution_age_group_two

# age_uniques_less_than_65 = less_than_65.groupby('RecordID')['Age'].first().reset_index()
percentage_group_two = (less_than_65.count() / age_uniques_train.count()) * 100
percentage_group_two

RecordID    45.104941
Age         45.104941
dtype: float64

<h3>Filtering only one height and one weight per patient</h3>

In [48]:
filtered_train_X = train_X[(train_X['Height'] != -1) & (train_X['Weight'] != -1) & (train_X['Height'].notna()) & (train_X['Weight'].notna())] 

In [49]:
height_uniques_train = filtered_train_X.groupby('RecordID')['Height'].first().reset_index() 
height_uniques_train.head()

,RecordID,Height
0,132543,180.3
1,132547,180.3
2,132548,162.6
3,132551,162.6
4,132555,175.3


In [50]:
weight_uniques_train = filtered_train_X.groupby('RecordID')['Weight'].first().reset_index() 
weight_uniques_train.head()

,RecordID,Weight
0,132543,84.6
1,132547,114.0
2,132548,87.0
3,132551,48.4
4,132555,66.1


<h3>Calculate the BMI</h3>

In [ ]:
height_uniques_train['Height'] = height_uniques_train['Height'] / 100 # Converting Height from cm to meters

# Creating a new DataFrame to store relevant data
#bmi_data = filtered_train_X[['RecordID', 'Height', 'Weight', 'BMI']].reset_index(drop=True)

#bmi_data = pd.concat([height_uniques_train['RecordID'], height_uniques_train['Height'], weight_uniques_train['Weight']], sort=True)
#bmi_data = pd.merge(height_uniques_train[['RecordID', 'Height']], weight_uniques_train[['RecordID', 'Weight']], on='RecordID')
#bmi_data.head()
#bmi_data['BMI'] = weight_uniques_train['Weight'] / (height_uniques_train['Height'] ** 2)


,RecordID,Height,Weight
0,132543,0.000180,84.6
1,132547,0.000180,114.0
2,132548,0.000163,87.0
3,132551,0.000163,48.4
4,132555,0.000175,66.1


<h1>Validation data</h1>

In [11]:
validation_X = physionet2012_dataset['val_X']

In [12]:
age_uniques_validation = validation_X.groupby('RecordID')['Age'].first().reset_index()
age_uniques_validation.count()

RecordID    1918
Age         1918
dtype: int64

<h1>Test data</h1>

In [14]:
test_X = physionet2012_dataset['test_X']

In [15]:
age_uniques_test = test_X.groupby('RecordID')['Age'].first().reset_index()
age_uniques_test.count()

RecordID    2399
Age         2399
dtype: int64